# Phi-4 Mini ONNX Parallel Function Calling Tutorial

Dieses Notebook zeigt, wie man Phi-4 Mini mit ONNX Runtime GenAI für parallele Funktionsaufrufe verwendet. Funktionsaufrufe ermöglichen es dem Modell, externe Tools und APIs basierend auf Benutzeranfragen intelligent zu nutzen.

## Überblick

In diesem Tutorial lernen Sie:
- Phi-4 Mini mit ONNX Runtime GenAI einzurichten
- Funktionsschemata für die Buchung von Flügen und Hotels zu definieren
- Geführte Generierung mit Lark-Grammatik für strukturierte Ausgaben zu verwenden
- Parallele Funktionsaufrufe für komplexe Reisebuchungsszenarien auszuführen

## Voraussetzungen

Bevor Sie dieses Notebook ausführen, stellen Sie sicher, dass Sie:
- Das Phi-4 Mini ONNX-Modell heruntergeladen haben
- Das Paket `onnxruntime-genai` installiert haben
- Grundlegendes Verständnis der Konzepte von Funktionsaufrufen besitzen


## Schritt 1: Erforderliche Bibliotheken importieren

Zuerst importieren wir die notwendigen Bibliotheken für die Implementierung unseres Funktionsaufrufs.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Schritt 2: Modellsetup und Konfiguration

Jetzt konfigurieren wir das Phi-4 Mini ONNX-Modell. Stellen Sie sicher, dass Sie den Pfad durch Ihr tatsächliches Modellverzeichnis ersetzen.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Schritt 3: Konfigurationsparameter festlegen

Richten Sie die Generierungsparameter ein, um das Ausgabeverhalten des Modells zu steuern. Diese Einstellungen gewährleisten deterministische und zielgerichtete Antworten für Funktionsaufrufe.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Schritt 4: Verfügbare Funktionen definieren

Hier definieren wir die Funktionen, die unser KI-Assistent aufrufen kann. In diesem Beispiel haben wir zwei reisebezogene Funktionen:
1. **booking_flight_tickets**: Zum Buchen von Flügen zwischen Flughäfen
2. **booking_hotels**: Zum Buchen von Hotelunterkünften

Die Funktionsdefinitionen folgen dem Schemaformat für Funktionsaufrufe von OpenAI.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Schritt 5: Hilfsfunktionen zur Generierung von Grammatik

Diese Hilfsfunktionen wandeln unsere Funktionsdefinitionen in das Lark-Grammatikformat um, das von ONNX Runtime GenAI für die geführte Generierung verwendet wird. Dies stellt sicher, dass das Modell gültige Funktionsaufrufe im korrekten JSON-Format ausgibt.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Schritt 6: Grammatikgenerierung testen

Lassen Sie uns unsere Funktionen zur Grammatikgenerierung testen, um zu sehen, wie sie unsere Werkzeugdefinitionen in das richtige Format umwandeln.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Schritt 7: System-Prompt und Generator vorbereiten

Jetzt erstellen wir den System-Prompt, der dem Modell die verfügbaren Tools beschreibt, und richten den Generator mit geführten Generierungsparametern ein.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Schritt 8: Generator mit geführter Generierung initialisieren

Jetzt erstellen wir den Generator mit unseren konfigurierten Parametern und wenden die Lark-Grammatik für die geführte Generierung an.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Schritt 9: Test der parallelen Funktionsaufrufe

Lassen Sie uns nun unsere Einrichtung mit einem komplexen Reisebuchungsszenario testen, das den Aufruf mehrerer Funktionen erfordert.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Generieren von Funktionsaufrufen

Das Modell wird nun strukturierte Funktionsaufrufe basierend auf der Anfrage des Nutzers erstellen. Dank der geführten Generierung wird die Ausgabe im gültigen JSON-Format vorliegen, das direkt ausgeführt werden kann.

**Erwartete Ausgabe**: Das Modell sollte Funktionsaufrufe generieren für:
1. `booking_flight_tickets` mit den Details von Peking (PEK) nach Paris (CDG)
2. `booking_hotels` mit Unterkunftsdetails in Paris

Führen Sie die Zelle unten aus, um die Live-Generierung zu sehen:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Fazit

🎉 **Herzlichen Glückwunsch!** Sie haben erfolgreich parallele Funktionsaufrufe mit Phi-4 Mini unter Verwendung von ONNX Runtime GenAI implementiert.

### Was Sie gelernt haben:

1. **Modellkonfiguration**: Wie man Phi-4 Mini mit ONNX Runtime GenAI einrichtet
2. **Funktionsdefinition**: Wie man Funktionsschemata für KI-Funktionsaufrufe definiert
3. **Geführte Generierung**: Wie man Lark-Grammatik für die strukturierte Ausgabe verwendet
4. **Parallele Funktionsaufrufe**: Wie man komplexe Anfragen mit mehreren Funktionsaufrufen verarbeitet

### Wichtige Vorteile:

- ✅ **Strukturierte Ausgabe**: Geführte Generierung sorgt für gültige JSON-Funktionsaufrufe
- ✅ **Parallele Verarbeitung**: Mehrere Funktionsaufrufe in einer einzigen Anfrage bearbeiten
- ✅ **Hohe Leistung**: ONNX Runtime bietet optimierte Inferenz
- ✅ **Flexibles Schema**: Einfaches Hinzufügen oder Ändern von Funktionsdefinitionen

### Ressourcen:

- [Phi-4 Mini Dokumentation](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI Dokumentation](https://onnxruntime.ai/docs/genai/)
- [Best Practices für Funktionsaufrufe](https://platform.openai.com/docs/guides/function-calling)



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
